# A Multi-hop QA Example with DSPy

We will do the below steps:

1. Load the Language Model and Retrieval Model
2. Load the "question-answer pairs" [HotPotQA](https://hotpotqa.github.io/) dataset to compile (train) the DsPy program
3. Build the signatures
4. Define the pipeline (as a module)
5. Define the evaluation metric
6. Compile (train) the pipeline with an optimizer (lets use ```BootstrapFewShot```)
7. Evaluate and compare the Compiled and Uncompiled pipeline

## 1 - Load the Language Model and Retriever and [Dataset](https://hotpotqa.github.io/)

In [1]:
import dspy
llama3_ollama = dspy.OllamaLocal(model='llama3:instruct')
# its very important to configure dspy with the LM
dspy.settings.configure(lm=llama3_ollama, max_tokens=4096, max_len=4096)

In [2]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=llama3_ollama, rm=colbertv2_wiki17_abstracts, max_tokens=1024)

In [3]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(
    train_seed=1, 
    train_size=30, 
    eval_seed=2023, 
    dev_size=50, 
    test_size=0
)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/home/templar/Desktop/projects/zaz/internal-projects/pocs/dspy-demo/venv/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(30, 50)

## 2 - Build Signature

In [4]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [5]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

## 3 - Define the Pipeline

In [6]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()
        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

## 4 - Define the Metric

In [7]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    # The predicted answer matches the gold answer.
    if not dspy.evaluate.answer_exact_match(example, pred): 
        return False
    # The retrieved context contains the gold answer.
    if not dspy.evaluate.answer_passage_match(example, pred): 
        return False

    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]
    # None of the generated queries exceeds 100 characters in length)
    if max([len(h) for h in hops]) > 100: 
        return False
        
    # None of the generated queries is roughly repeated 
    # (i.e., none is within 0.8 or higher F1 score of earlier queries).
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): 
        return False
        
    return True

## 5 - Compile the DSPy Program/pipeline (train)

In [8]:
%%time

from dspy.teleprompt import BootstrapFewShot

# Define Optimizer
optimizer = BootstrapFewShot(metric=validate_context_and_answer_and_hops)

# Compile
compiled_baleen = optimizer.compile(
    SimplifiedBaleen(), 
    teacher=SimplifiedBaleen(passages_per_hop=2), 
    trainset=trainset
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [10:13<00:00, 20.46s/it]

CPU times: user 744 ms, sys: 163 ms, total: 907 ms
Wall time: 10min 14s


## 6 - Evaluate compiled and uncompiled pipelines

### 6.1 - Uncompiled Example

In [9]:
# Ask any question you like to this simple RAG program.
my_question = "How many storeys are in the castle that David Gregory inherited?"

# Get the prediction from uncompiled Baleen
uncompiled_baleen = SimplifiedBaleen()
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: How many storeys are in the castle that David Gregory inherited?
Predicted Answer: 5
Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'David Gregory (historian) | David Gregory (1696–1767) was an English churchman and academic, Dean of Christ Church, Oxford and the first Regius Professor of Modern History at Oxford....', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...', 'Kinnairdy Castle | Kinnairdy Castle is a tower house, having five storeys and a garret, two miles south of Aberchirder, Aberdeenshire, Scotland. The alternative name is Old Kinnairdy....', 'Kinnaird Castle, Brechin | Kinnaird Castle is a 15th-century castle in Angus, 

### 6.2 - Compiled Example

In [10]:
# Ask any question you like to this simple RAG program.
my_question = "How many storeys are in the castle that David Gregory inherited?"

# Get the prediction from uncompiled Baleen
pred_compiled = compiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred_compiled.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred_compiled.context]}")

Question: How many storeys are in the castle that David Gregory inherited?
Predicted Answer: 5
Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'David Gregory (historian) | David Gregory (1696–1767) was an English churchman and academic, Dean of Christ Church, Oxford and the first Regius Professor of Modern History at Oxford....', 'David Gregory (mathematician) | David Gregory (originally spelt Gregorie) FRS (? 1659 – 10 October 1708) was a Scottish mathematician and astronomer. He was professor of mathematics at the University ...', 'Kinnairdy Castle | Kinnairdy Castle is a tower house, having five storeys and a garret, two miles south of Aberchirder, Aberdeenshire, Scotland. The alternative name is Old Kinnairdy....', 'Kinnaird Castle, Brechin | Kinnaird Castle is a 15th-century castle in Angus, 

### 6.3 - Performance Comparison between Compiled and Uncompiled

In [11]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))
    return gold_titles.issubset(found_titles)

In [12]:
from dspy.evaluate.evaluate import Evaluate
# Set up the `evaluate_on_hotpotqa` function.
evaluate_on_hotpotqa = Evaluate(
    devset=devset, 
    num_threads=1, 
    display_progress=True, 
    display_table=5
)

In [13]:
# Evaluate the uncompiled Baleen pipeline
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(
    uncompiled_baleen, 
    metric=gold_passages_retrieved
)
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")

Average Metric: 15 / 50  (30.0): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [16:22<00:00, 19.65s/it]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","[""Marc-André Fleury | Marc-André Fleury (born November 28, 1984) is a French-Canadian professional ice hockey goaltender playing for the Vegas Golden Knights of the National...",NHL,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['List of Detroit Red Wings general managers | The Detroit Red Wings are a professional ice hockey team based in Detroit, Michigan. They are members...",Steve Yzerman.,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","['Kilmarnock | Kilmarnock (Scottish Gaelic: ""Cille Mheàrnaig"" , ""Meàrnag\'s church"") is a large burgh in East Ayrshire, Scotland with a population of 46,350, making it...",Irvine,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith.', '10th century | The...",Alfred,False


## Retrieval Score for uncompiled Baleen: 30.0


In [14]:
# Evaluate the compiled Baleen pipeline
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(
    compiled_baleen, 
    metric=gold_passages_retrieved
)
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

Average Metric: 15 / 50  (30.0): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [16:45<00:00, 20.11s/it]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...","Context: [1] ""Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People's Republic of China..."" Question: Are both Cangzhou and Qionghai in...",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017 NHL Expansion Draft', '2017–18 Pittsburgh Penguins season'}","[""Marc-André Fleury | Marc-André Fleury (born November 28, 1984) is a French-Canadian professional ice hockey goaltender playing for the Vegas Golden Knights of the National...",NHL,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['List of Detroit Red Wings general managers | The Detroit Red Wings are a professional ice hockey team based in Detroit, Michigan. They are members...",Steve,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","['Kilmarnock | Kilmarnock (Scottish Gaelic: ""Cille Mheàrnaig"" , ""Meàrnag\'s church"") is a large burgh in East Ayrshire, Scotland with a population of 46,350, making it...","Context: [1] ""Kilmarnock | Kilmarnock (Scottish Gaelic: ""Cille Mheàrnaig"", ""Meàrnag's church"") is a large burgh in East Ayrshire, Scotland with a population of 46,350, making...",False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith.', '10th century | The...",Alfred,False


## Retrieval Score for compiled Baleen: 30.0


In [17]:
# Save the compiled pipeline
compiled_baleen.save("./saved_pipeline")